In [1]:
# Machine learning stratification

In [2]:
# Importar librerias necesarias
import pandas as pd
import numpy as np
import time
from itertools import product
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, KernelPCA
from sklearn.cluster import KMeans, MiniBatchKMeans, AgglomerativeClustering, Birch, SpectralClustering
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, davies_bouldin_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import friedmanchisquare
import random
import tensorflow as tf
from google.colab import files

In [3]:
# Instalar la libreria que permite leer archivos en formato .rds
!pip install pyreadr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.3/418.3 kB 7.9 MB/s eta 0:00:00


In [4]:
from google.colab import files
uploaded = files.upload()

Saving 9_PSU_indicators_Cuenca.rds to 9_PSU_indicators_Cuenca.rds


In [5]:
# Dataset de las UPM de Cuenca
import pyreadr
# Leer archivo .rds
result = pyreadr.read_r('9_PSU_indicators_Cuenca.rds')
# Extraer el dataframe
df = result[None]
# Ver los datos
df.head()

,id_upm,area,dom_area,p_educ_superior,p_pers_ocupad,p_homb_ocupad,p_muje_ocupad,p_no_ocupad_15_17,p_pers_no_desocupad,p_homb_no_desocupad,...,v_dispone_luz,h_serv_higienico_es,h_lugar_cocinar,h_dispone_ducha,h_dispone_telef,h_dispone_celular,h_dispone_internet,h_dispone_compu,h_dispone_tvcable,h_todos_bienes
0,010150001001,1,271,23.32,61.67,72.73,52.26,82.35,94.65,92.31,...,100.00,82.61,95.65,79.13,32.17,84.35,7.83,45.22,0.00,0.00
1,010150001002,1,271,18.18,66.32,77.78,56.31,67.57,93.77,93.96,...,100.00,71.34,85.99,69.43,14.01,69.43,11.46,35.67,1.27,0.00
2,010150001003,1,271,33.53,67.04,79.55,54.81,72.00,98.35,98.13,...,100.00,91.30,93.48,83.70,27.17,59.78,8.70,38.04,3.26,2.17
3,010150001004,1,271,14.71,64.74,78.72,53.22,79.07,99.02,99.11,...,99.15,78.99,94.12,74.79,29.41,81.51,10.08,43.70,2.52,0.84
4,010150001005,1,271,23.90,60.57,72.22,50.87,93.02,95.05,94.55,...,99.12,90.52,89.66,82.76,41.38,63.79,4.31,46.55,3.45,0.00


In [6]:
# Escalar los datos
df_sel = df.iloc[:, 3:43]
X = df_sel.values
X = X.astype(np.float32)
print(X)
print(X.shape)

[[23.32 61.67 72.73 ... 45.22  0.    0.  ]
 [18.18 66.32 77.78 ... 35.67  1.27  0.  ]
 [33.53 67.04 79.55 ... 38.04  3.26  2.17]
 ...
 [ 5.17 64.   75.58 ... 31.88  4.35  0.  ]
 [13.87 70.05 87.78 ... 22.54  0.    0.  ]
 [ 8.11 58.39 74.79 ... 31.    2.    1.  ]]
(781, 40)


In [7]:
# Definicion de funciones
# Diccionario de configuracion de hiperparametros

config_dict = {
    "representations": [
        {"name": "Original"},
        {"name": "PCA", "n_components": [2, 3, 5]},
        {"name": "Kernel PCA", "n_components": [2], "kernel": ["rbf", "poly"], "gamma": [0.1, 1]},
        {"name": "Autoencoder", "encoding_dim": [2, 3], "layers": [(64,), (128, 64)], "dropout": [0.0, 0.2], "epochs": [50], "batch_size": [32]}
    ],
    "clustering": [
        {"name": "KMeans", "n_clusters": [3, 4, 5], "n_init": [10, 20]},
        {"name": "MiniBatchKMeans", "n_clusters": [3, 4, 5], "batch_size": [50, 100]},
        {"name": "Agglomerative", "n_clusters": [3, 4, 5], "linkage": ["ward", "average"]},
        {"name": "BIRCH", "n_clusters": [3, 4, 5], "threshold": [0.5, 1.0]},
        {"name": "GaussianMixture", "n_components": [3, 4, 5], "covariance_type": ["full", "diag"]},
        {"name": "SpectralClustering", "n_clusters": [3, 4, 5], "gamma": [0.5, 1.0]}
    ]
}

# Funcion de transformacion Autoencoder

def autoencoder_custom(X, encoding_dim=2, layers=(64,), dropout=0.0, epochs=50, batch_size=32):
    input_dim = X.shape[1]
    input_layer = Input(shape=(input_dim,))
    x = input_layer
    for units in layers:
        x = Dense(units, activation='relu')(x)
        if dropout > 0.0:
            x = Dropout(dropout)(x)
    encoded = Dense(encoding_dim, activation='relu')(x)
    decoded = Dense(input_dim, activation='linear')(encoded)

    autoencoder = Model(input_layer, decoded)
    encoder = Model(input_layer, encoded)

    autoencoder.compile(optimizer=Adam(), loss='mse')
    autoencoder.fit(X, X, epochs=epochs, batch_size=batch_size, verbose=0)

    return encoder, encoder.predict(X)

# Representaciones

def get_representations(X, rep_conf):
    name = rep_conf["name"]
    if name == "Original":
        return {"Original": (X, {}, lambda X_new: X_new)}
    elif name == "PCA":
        valid_components = [n for n in rep_conf["n_components"] if n <= X.shape[1]]
        return {
            f"PCA (n={n})": (PCA(n_components=n).fit(X).transform(X), {"n_components": n}, PCA(n_components=n).fit(X).transform)
            for n in valid_components
        }
    elif name == "Kernel PCA":
        valid_components = [n for n in rep_conf["n_components"] if n <= X.shape[1]]
        return {
            f"KernelPCA (kernel={k}, gamma={g})": (
                KernelPCA(n_components=n, kernel=k, gamma=g).fit(X).transform(X),
                {"n_components": n, "kernel": k, "gamma": g},
                KernelPCA(n_components=n, kernel=k, gamma=g).fit(X).transform
            )
            for n in valid_components for k in rep_conf["kernel"] for g in rep_conf["gamma"]
        }
    elif name == "Autoencoder":
        result = {}
        for d in rep_conf["encoding_dim"]:
            for layers in rep_conf["layers"]:
                for dropout in rep_conf["dropout"]:
                    encoder, X_encoded = autoencoder_custom(X, encoding_dim=d, layers=layers, dropout=dropout,
                                                             epochs=rep_conf["epochs"][0], batch_size=rep_conf["batch_size"][0])
                    result[f"Autoencoder (dim={d}, layers={layers}, dropout={dropout})"] = (
                        X_encoded,
                        {"encoding_dim": d, "layers": layers, "dropout": dropout, "epochs": rep_conf["epochs"][0], "batch_size": rep_conf["batch_size"][0]},
                        encoder.predict
                    )
        return result
    return {}

# Métodos de clustering

def get_clustering_methods(clust_conf):
    methods = []
    name = clust_conf["name"]

    if name == "KMeans":
        for k, n_init in product(clust_conf["n_clusters"], clust_conf["n_init"]):
            def func(X, k=k, n_init=n_init, return_model=False):
                model = KMeans(n_clusters=k, n_init=n_init, random_state=42).fit(X)
                return model if return_model else model.labels_
            methods.append((f"KMeans (k={k})", func, {"n_clusters": k, "n_init": n_init}))

    elif name == "MiniBatchKMeans":
        for k, b in product(clust_conf["n_clusters"], clust_conf["batch_size"]):
            def func(X, k=k, b=b, return_model=False):
                model = MiniBatchKMeans(n_clusters=k, batch_size=b, random_state=42).fit(X)
                return model if return_model else model.labels_
            methods.append((f"MiniBatchKMeans (k={k})", func, {"n_clusters": k, "batch_size": b}))

    elif name == "Agglomerative":
        for k, linkage in product(clust_conf["n_clusters"], clust_conf["linkage"]):
            def func(X, k=k, linkage=linkage, return_model=False):
                model = AgglomerativeClustering(n_clusters=k, linkage=linkage).fit(X)
                return model if return_model else model.labels_
            methods.append((f"Agglomerative (k={k}, linkage={linkage})", func, {"n_clusters": k, "linkage": linkage}))

    elif name == "BIRCH":
        for k, t in product(clust_conf["n_clusters"], clust_conf["threshold"]):
            def func(X, k=k, t=t, return_model=False):
                model = Birch(n_clusters=k, threshold=t).fit(X)
                return model if return_model else model.labels_
            methods.append((f"BIRCH (k={k}, t={t})", func, {"n_clusters": k, "threshold": t}))

    elif name == "GaussianMixture":
        for k, cov in product(clust_conf["n_components"], clust_conf["covariance_type"]):
            def func(X, k=k, cov=cov, return_model=False):
                model = GaussianMixture(n_components=k, covariance_type=cov, random_state=42).fit(X)
                return model if return_model else model.predict(X)
            methods.append((f"GMM (k={k}, cov={cov})", func, {"n_components": k, "covariance_type": cov}))

    elif name == "SpectralClustering":
        for k, g in product(clust_conf["n_clusters"], clust_conf["gamma"]):
            def func(X, k=k, g=g, return_model=False):
                model = SpectralClustering(n_clusters=k, affinity='rbf', gamma=g, assign_labels='kmeans', random_state=42).fit(X)
                return model if return_model else model.labels_
            methods.append((f"Spectral (k={k}, gamma={g})", func, {"n_clusters": k, "gamma": g}))

    return methods

# Funcion de ejecución final

def run_experiments(X, config_dict):
    all_results = []
    label_results = {}
    label_columns = {}
    cv = KFold(n_splits=5, shuffle=True, random_state=42)

    for fold_idx, (train_idx, test_idx) in enumerate(cv.split(X)):
        X_train, X_test = X[train_idx], X[test_idx]

        for rep_conf in config_dict['representations']:
            representations = get_representations(X_train, rep_conf)

            for rep_name, (X_rep, rep_params, transformer) in representations.items():
                X_test_rep = transformer(X_test)
                for clust_conf in config_dict['clustering']:
                    methods = get_clustering_methods(clust_conf)

                    for method_name, cluster_func, clust_params in methods:
                        start = time.time()
                        try:
                            model = cluster_func(X_rep, return_model=True)
                            labels_train = model.labels_ if hasattr(model, "labels_") else model.predict(X_rep)
                            labels_test = model.predict(X_test_rep) if hasattr(model, "predict") else cluster_func(np.vstack([X_rep, X_test_rep]))[len(train_idx):]

                            elapsed = time.time() - start

                            silhouette = silhouette_score(X_rep, labels_train)
                            davies = davies_bouldin_score(X_rep, labels_train)

                            all_results.append({
                                'Representación': rep_name,
                                'Método': method_name,
                                'Tiempo (s)': elapsed,
                                'Silhouette': silhouette,
                                'Davies-Bouldin': davies,
                                'Params_Representación': rep_params,
                                'Params_Clustering': clust_params
                            })

                            key = f"{rep_name} + {method_name}"
                            if key not in label_columns:
                                label_columns[key] = np.full(len(X), np.nan)
                            label_columns[key][train_idx] = labels_train
                            label_columns[key][test_idx] = labels_test

                            labels_full = np.empty(len(X))
                            labels_full[train_idx] = labels_train
                            labels_full[test_idx] = labels_test
                            label_results[(rep_name, method_name, str(start))] = labels_full

                        except Exception as e:
                            elapsed = time.time() - start
                            print(f"Error en {rep_name} + {method_name}: {e}")
                            all_results.append({
                                'Representación': rep_name,
                                'Método': method_name,
                                'Tiempo (s)': elapsed,
                                'Silhouette': None,
                                'Davies-Bouldin': None,
                                'Error': str(e),
                                'Params_Representación': rep_params,
                                'Params_Clustering': clust_params
                            })

    # Guardar resultados
    final_labels = pd.DataFrame(label_columns)
    final_labels.to_csv("etiquetas_finales_cross_validation.csv")

    df = pd.DataFrame(all_results)
    df.to_csv("todos_los_resultados.csv", index=False)

    # Métricas agregadas
    df_validos = df.dropna(subset=['Silhouette', 'Davies-Bouldin'])
    resumen_metricas = df_validos.groupby(['Representación', 'Método']).agg({
        'Silhouette': ['mean', 'std', lambda x: np.std(x) / np.mean(x) if np.mean(x) != 0 else 0],
        'Davies-Bouldin': ['mean', 'std', lambda x: np.std(x) / np.mean(x) if np.mean(x) != 0 else 0],
        'Tiempo (s)': ['mean', 'std']
    }).reset_index()
    resumen_metricas.columns = ['Representación', 'Método', 'Silhouette_Media', 'Silhouette_STD', 'Silhouette_CV',
                                'DB_Media', 'DB_STD', 'DB_CV', 'Tiempo_Medio', 'Tiempo_STD']

    # Normalización MinMax
    def min_max_normalize(series):
        if series.max() == series.min():
            return np.zeros_like(series)
        return (series - series.min()) / (series.max() - series.min())

    resumen_metricas['Silhouette_Norm'] = min_max_normalize(resumen_metricas['Silhouette_Media'])
    resumen_metricas['DB_Norm'] = min_max_normalize(resumen_metricas['DB_Media'])
    resumen_metricas['Score_Norm_Global'] = resumen_metricas['Silhouette_Norm'] - resumen_metricas['DB_Norm']
    resumen_metricas.to_csv("resumen_metricas_clustering.csv", index=False)

    # Score normalizado por Método
    score_por_metodo = []
    for method in resumen_metricas['Método'].unique():
        sub = resumen_metricas[resumen_metricas['Método'] == method].copy()
        sub['Silhouette_Norm'] = min_max_normalize(sub['Silhouette_Media'])
        sub['DB_Norm'] = min_max_normalize(sub['DB_Media'])
        sub['Score_Norm_Por_Metodo'] = sub['Silhouette_Norm'] - sub['DB_Norm']
        score_por_metodo.append(sub)

    score_metodo_df = pd.concat(score_por_metodo, ignore_index=True)
    score_metodo_df.to_csv("score_normalizado_por_metodo.csv", index=False)

    return df, label_results, resumen_metricas

# Resumenes

def resumen_tiempos(df_resultados):
    resumen = df_resultados.dropna(subset=['Tiempo (s)']).groupby(['Representación', 'Método'])['Tiempo (s)'].agg(['mean', 'std']).reset_index()
    resumen.columns = ['Representación', 'Método', 'Tiempo Medio (s)', 'Desviación Std (s)']
    return resumen


In [8]:
# Ejecucion del calculo y obtencion de resultados
# Uso ejecutable final
if __name__ == "__main__":
    np.random.seed(42)
    random.seed(42)
    tf.random.set_seed(42)

    df_resultados, etiquetas, resumen_metricas = run_experiments(X, config_dict)
    resumen = resumen_tiempos(df_resultados)

    df_resultados.to_csv("resultados_validacion_cruzada.csv", index=False)
    resumen.to_csv("resumen_tiempos.csv", index=False)

    params_df = df_resultados[["Representación", "Método", "Params_Representación", "Params_Clustering"]]
    params_df.to_csv("hiperparametros_validacion_cruzada.csv", index=False)

    files.download("resultados_validacion_cruzada.csv")
    files.download("resumen_tiempos.csv")
    files.download("hiperparametros_validacion_cruzada.csv")
    files.download("resumen_metricas_clustering.csv")
    files.download("todos_los_resultados.csv")
    files.download("etiquetas_finales_cross_validation.csv")
    files.download("score_normalizado_por_metodo.csv")

    print("✔ Resultados guardados y descargados exitosamente.")

/usr/local/lib/python3.12/dist-packages/sklearn/manifold/_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/manifold/_spectral_embedding.py:455: UserWarning: Exited at iteration 784 with accuracies 
[2.30284817e-05 9.26917917e-06 9.20237786e-06 6.96791734e-05]
not reaching the requested tolerance 9.298324584960938e-06.
Use iteration 710 instead with accuracy 
2.7355038093156984e-05.

  _, diffusion_map = lobpcg(
/usr/local/lib/python3.12/dist-packages/sklearn/manifold/_spectral_embedding.py:455: UserWarning: Exited postprocessing with accuracies 
[1.95574422e-05 8.98458226e-06 9.92881603e-06 7.09493156e-05]
not reaching the requested tolerance 9.298324584960938e-06.
  _, diffusion_map = lobpcg(
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:188: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/usr/local/lib/py

Error en Original + Spectral (k=3, gamma=0.5): Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)


/usr/local/lib/python3.12/dist-packages/sklearn/manifold/_spectral_embedding.py:455: UserWarning: Exited at iteration 53 with accuracies 
[3.36131593e-06 1.54743927e-05 7.89642254e-06 1.59632880e-04]
not reaching the requested tolerance 9.298324584960938e-06.
Use iteration 53 instead with accuracy 
4.6591252877389725e-05.

  _, diffusion_map = lobpcg(
/usr/local/lib/python3.12/dist-packages/sklearn/manifold/_spectral_embedding.py:455: UserWarning: Exited postprocessing with accuracies 
[3.36131591e-06 1.54743927e-05 7.89642252e-06 1.59632880e-04]
not reaching the requested tolerance 9.298324584960938e-06.
  _, diffusion_map = lobpcg(
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:188: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:199: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
/usr/local/lib/python3.1

Error en Original + Spectral (k=3, gamma=1.0): Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)


/usr/local/lib/python3.12/dist-packages/sklearn/manifold/_spectral_embedding.py:455: UserWarning: Exited at iteration 479 with accuracies 
[4.20624403e-06 7.08531718e-06 1.97925124e-05 6.88012690e-06
 6.41531314e-05]
not reaching the requested tolerance 9.298324584960938e-06.
Use iteration 479 instead with accuracy 
2.0423466394864472e-05.

  _, diffusion_map = lobpcg(
/usr/local/lib/python3.12/dist-packages/sklearn/manifold/_spectral_embedding.py:455: UserWarning: Exited postprocessing with accuracies 
[4.20624420e-06 7.08531701e-06 1.97925017e-05 6.88015778e-06
 6.41531314e-05]
not reaching the requested tolerance 9.298324584960938e-06.
  _, diffusion_map = lobpcg(
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:188: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/usr/local/lib/python3.12/dist-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/usr/local/lib/python3.12/dist-packages/skle

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✔ Resultados guardados y descargados exitosamente.
